In [7]:
import pandas as pd
import requests
import os
import geopandas as gpd
import zipfile

In [2]:
#Crea directorio "capas" para descarga de archivos
if not os.path.exists("marco"):
    os.makedirs("marco")

In [3]:
#Descargar y unzip de marco geoestadístico 2024

url="https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/marcogeo/794551132173/mg_2024_integrado.zip"
response = requests.get(url)
with open("marco/mg_2024_integrado.zip", "wb") as f:
    f.write(response.content)

In [8]:
#Descomprimir
with zipfile.ZipFile("marco/mg_2024_integrado.zip", "r") as zip_ref:
    zip_ref.extractall("marco")

#Eliminar el zip
os.remove("marco/mg_2024_integrado.zip")


In [41]:
##Leer entidades y municipios

ent=gpd.read_file("marco/conjunto_de_datos/00ent.shp")
ent.columns=ent.columns.str.lower()
#Dejar solo cve_ent y nomgeo
ent=ent[["cve_ent","nomgeo"]]
#Cambiar nomgeo por nom_ent
ent=ent.rename(columns={"nomgeo":"nom_ent"})

mun=gpd.read_file("marco/conjunto_de_datos/00mun.shp")
mun.columns= mun.columns.str.lower()
mun=mun.rename(columns={"nomgeo":"nom_mun"})
#Pegar nom_ent a mun
mun=mun.merge(ent, on="cve_ent", how="left")
print(mun.crs)
mun

EPSG:6372


,cvegeo,cve_ent,cve_mun,nom_mun,geometry,nom_ent
0,01005,01,005,Jesús María,"POLYGON ((2463657.764 1116053.82, 2463738.92 1...",Aguascalientes
1,01008,01,008,San José de Gracia,"POLYGON ((2453326.553 1143576.423, 2453363.325...",Aguascalientes
2,01001,01,001,Aguascalientes,"POLYGON ((2488980.227 1117358.447, 2489013.495...",Aguascalientes
3,01004,01,004,Cosío,"POLYGON ((2469550.852 1159302.643, 2469738.209...",Aguascalientes
4,01011,01,011,San Francisco de los Romo,"POLYGON ((2483638.349 1120034.022, 2483946.175...",Aguascalientes
...,...,...,...,...,...,...
2473,32007,32,007,Concepción del Oro,"POLYGON ((2566164.344 1419182.916, 2569089.16 ...",Zacatecas
2474,32047,32,047,Teúl de González Ortega,"POLYGON ((2348121.417 1055521.188, 2348299.263...",Zacatecas
2475,32012,32,012,Genaro Codina,"POLYGON ((2428441.5 1178012.694, 2428503.511 1...",Zacatecas
2476,32006,32,006,Cañitas de Felipe Pescador,"POLYGON ((2419835.035 1295047.92, 2420029.299 ...",Zacatecas


In [30]:
# Carpeta con archivos
carpeta = "capas"
list_archivos = os.listdir(carpeta)
list_archivos

['p_1.pmtiles',
 'p_10.pmtiles',
 'p_11.pmtiles',
 'p_12.pmtiles',
 'p_2.pmtiles',
 'p_3.pmtiles',
 'p_4.pmtiles',
 'p_5.pmtiles',
 'p_6.pmtiles',
 'p_7.pmtiles',
 'p_8.pmtiles',
 'p_9.pmtiles']

In [31]:
sectores_dict = {
    "p_1": "semicon,electromovilidad,bienes",
    "p_2": "farma,petroquimica",
    "p_3": "agro,textil,electromovilidad,bienes,farma,petroquimica,aeroespacial",
    "p_4": "bienes",
    "p_5": "electromovilidad,bienes,semicon,aeroespacial",
    "p_6": "textil,agro,electromovilidad,bienes",
    "p_7": "textil,agro,electromovilidad,bienes,farma,semicon,petroquimica",
    "p_8": "textil,agro,farma,petroquimica,aeroespacial",
    "p_9": "textil,agro,electromovilidad,bienes,petroquimica,aeroespacial",
    "p_10": "agro,bienes,aeroespacial",
    "p_11": "agro,bienes",
    "p_12": "agro,aeroespacial"
}

In [42]:
# Carpeta con archivos

list_gdf = []

# Recorrer archivos
for archivo in list_archivos:
    if archivo.endswith(".pmtiles"):
        gdf = gpd.read_file(os.path.join(carpeta, archivo))

        # Quitar cve_sector si existe
        if 'cve_sector' in gdf.columns:
            gdf = gdf.drop(columns='cve_sector')

        # Determinar clave del archivo
        clave = archivo.split(".")[0]

        # Caso especial para p_10
        if clave == "p_10":
            columnas = ['geometry', 'mvt_id'] if 'mvt_id' in gdf.columns else ['geometry']
            gdf = gdf[columnas]
            gdf['corredor'] = "Golfo"
            gdf['id'] = 10
            gdf['nombre'] = "Golfo"

        if clave == "p_4":
            gdf['corredor'] = "Durango"
            gdf['id'] = 4
            gdf['nombre'] = "Durango"

        if clave == "p_5":
            gdf['corredor'] = "Noroeste"
            gdf['id'] = 5
            gdf['nombre'] = "Noroeste"

        # Agregar o corregir sector_estrategico si está en el diccionario
        if clave in sectores_dict:
            gdf['sector_estrategico'] = sectores_dict[clave]

        list_gdf.append(gdf)

# Concatenar todo
poligonos = pd.concat(list_gdf, ignore_index=True)
#Eliminar si la geografía es un punto
poligonos = poligonos[poligonos.geometry.type != 'Point']
poligonos = poligonos.to_crs(epsg=3857)
#Hacer disolve para eliminar malla
poligonos = poligonos.dissolve(by="id", aggfunc="first")
#Cambiar crs a la de la capa de poligonos
poligonos = poligonos.to_crs(mun.crs)
print(poligonos.crs)

EPSG:6372


In [53]:
#Hacer una unión de los polígonos con los municipios para saber donde están
union=mun.sjoin(poligonos,how="left")
#Eliminar geometría y transformar a dataframe
union = pd.DataFrame(union.drop(columns='geometry'))
#Eliminar mvt_id
union=union.drop(columns={'mvt_id','nombre'})
union

,cvegeo,cve_ent,cve_mun,nom_mun,nom_ent,id,corredor,sector_estrategico
0,01005,01,005,Jesús María,Aguascalientes,6,Bajío,"textil,agro,electromovilidad,bienes"
1,01008,01,008,San José de Gracia,Aguascalientes,6,Bajío,"textil,agro,electromovilidad,bienes"
1,01008,01,008,San José de Gracia,Aguascalientes,4,Durango,bienes
2,01001,01,001,Aguascalientes,Aguascalientes,6,Bajío,"textil,agro,electromovilidad,bienes"
2,01001,01,001,Aguascalientes,Aguascalientes,7,Pacífico,"textil,agro,electromovilidad,bienes,farma,semi..."
...,...,...,...,...,...,...,...,...
2475,32012,32,012,Genaro Codina,Zacatecas,6,Bajío,"textil,agro,electromovilidad,bienes"
2475,32012,32,012,Genaro Codina,Zacatecas,4,Durango,bienes
2476,32006,32,006,Cañitas de Felipe Pescador,Zacatecas,4,Durango,bienes
2477,32011,32,011,Trinidad García de la Cadena,Zacatecas,7,Pacífico,"textil,agro,electromovilidad,bienes,farma,semi..."


In [54]:
#Salvar csv
union.to_csv("municipios_polos.csv",index=False, encoding='latin1')